In [1]:
from mlcomp.data.load import load_regression_train
from lenskit.algorithms import als, funksvd
from lenskit import batch, crossfold as xf
from lenskit.metrics.predict import rmse
from ConfigSpace import Configuration, ConfigurationSpace
from smac import Scenario, HyperparameterOptimizationFacade
from pathlib import Path

In [2]:
df = load_regression_train()

# remove everything before 01-01-2014
df = df.loc[df['timestamp'] > 1388530800 * 1000]

In [3]:
# generate train test split
ttpair_gen = xf.partition_users(df, 2, xf.SampleFrac(0.15))
ttpair = ttpair_gen.__next__()

train = ttpair.train
test = ttpair.test


X_val = test.drop(columns=['rating'])
y_val = test['rating']

In [4]:
def train_fn(config: Configuration, seed: int) -> float:
    
    model = als.BiasedMF(
        features=config['features'],
        iterations=config['iterations'],
        reg=config['reg'],
        damping=config['damping'],
        rng_spec=seed
    )
    model.fit(train)

    pred = batch.predict(model, X_val)

    # lenskit messed up the index but keeps the order, so we can fix it like this:
    pred.index = X_val.index

    return rmse(pred['prediction'], y_val) 


In [ ]:
cs = ConfigurationSpace({
    'features': (10, 100),
    'iterations': (10, 100),
    'reg': (0.001, 0.1),
    'damping': (1, 8),
})

scenario = Scenario(
    configspace=cs,
    n_trials=400,
    walltime_limit=3600,
    n_workers=2,
    output_directory=Path(f"smac3_BiasedMF"),
)

smac = HyperparameterOptimizationFacade(scenario=scenario, target_function=train_fn)

incumbent = smac.optimize()

In [6]:
incumbent

Configuration(values={
  'damping': 5,
  'features': 67,
  'iterations': 23,
  'reg': 0.09358052502945066,
})

### Doc


#### Run 1:
- BiasedMF
- CS:
  ```Python
  ConfigurationSpace({
      'features': (10, 100),
      'iterations': (10, 100),
      'reg': (0.001, 0.1),
      'damping': (1, 8),
  })
  ```
- Stopped because of time limit
- 339/400 trials left
- Config:
  ```Python
  Configuration(values={
    'damping': 5,
    'features': 67,
    'iterations': 23,
    'reg': 0.09358052502945066,
  })
  ```